## 7.2 使用Keras回调函数和TensorBoard来检查并监控深度学习模型
### 7.2.1 训练过程中将回调函数作用于模型
**1. ModelCheckpoint与EarlyStopping回调函数**

In [ ]:
import keras

# 通过fit的callbakcs参数将回调函数传入模型中，这个参数
# 接收一个回调函数的列表。可以传入任意个数的回调函数
callbacks_list = [
    # 如果不再改善，就中断训练
    keras.callbacks.EarlyStopping(
        monitor='acc',  # 监控模型的验证精度
        patience=1,     # 如果精度在多于一轮的时间（即两轮）内不再改善，中断训练
    ),
    # 在每轮过后保存当前权重
    keras.callbacks.ModelCheckpoint(
        filepath='my_model.h5',     # 目标模型文件的保存路径
        # 下面两行参数的含义是，如果val_loss没有改善，那么不需要覆盖模型文件。
        # 这样就可以始终保存在训练过程中见到的最佳模型
        monitor='val_loss',
        save_best_only=True,
    )
]
'''
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])      # 监控精度，应该是模型指标的一部分

# 注意，由于回调函数要监控验证损失和验证精度，所以在调用fit时需要传入validation_data（验证数据）
model.fit(x, y,
          epochs=10,
          batch_size=32,
          callbacks=callbacks_list,
          validation_data=(x_val, y_val))
'''          

**2. ReduceLROnPlateau 回调函数**

In [ ]:
callbacks_list = [
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',     # 监控模型的验证损失
        factor=0.1,             # 触发时将学习率除以10
        patience=10,            # 如果验证损失在10轮内都没有改善，那么就触发这个回调函数
    )
]
'''
# 注意，由于回调函数要监控验证损失，所以在调用fit时需要传入validation_data（验证数据）
model.fit(x, y,
          epochs=10,
          batch_size=32,
          callbacks=callbacks_list,
          validation_data=(x_val, y_val))
'''   

**3. 编写你自己的回调函数**

In [ ]:
import keras
import numpy as np

class ActivationLogger(keras.callbacks.Callback):

    def set_model(self, model):
        # 在训练之前由父模型调用，告诉回调函数是哪个模型在调用它
        self.model = model
        layer_outputs = [layer.output for layer in model.layers]
        # 模型实例，返回每层的激活
        self.activation_model = keras.models.Model(model.input, layer_outputs)
    
    def on_epoch_end(self, epoch, logs=None):
        if self.validation_data is None:
            raise RuntimeError('Requires validation_data.')
        # 获取验证数据的第一个输入样本
        validation_sample = self.validation_data[0][0:1]
        activations = self.activation_model.predict(validation_sample)
        # 将数组保存到硬盘
        f = open('activations_at_epoch_' + str(epoch) + '.npz', 'w')
        np.savez(f, activations)
        f.close()

### 7.2.2 TensorBoard简介：TensorFlow的可视化框架
**代码清单 7-7** 使用了TensorBoard的文本分类模型

In [ ]:
import keras
from keras import layers
from keras.datasets import imdb
from keras.preprocessing import sequence

import numpy as np
# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a, **k: np_load_old(*a, allow_pickle=True, **k)

# 作为特征的单词个数
max_features = 2000
# 在maxlen个单词之后截断文本（这些单词都属于前max_features个最常见的单词）
max_len = 500

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

# restore np.load for future normal usage
np.load = np_load_old

x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

model = keras.models.Sequential()
model.add(layers.Embedding(max_features, 128,
                           input_length=max_len,
                           name='embed'))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPool1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(1))
model.summary()
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])                           

**代码清单 7-9** 使用一个TensorBoard回调函数来训练模型

In [ ]:
callbacks = [
    keras.callbacks.TensorBoard(
        log_dir='my_log_dir',   # 日志文件将被写入这个位置
        histogram_freq = 1,     # 每一轮之后记录激活直方图
        embeddings_freq=1,      # 每一轮之后记录嵌入数据
    )
]

history = model.fit(x_train, y_train,
                    epochs=20,
                    batch_size=128,
                    validation_split=0.2,
                    callbacks=callbacks)

In [ ]:
from keras.utils import plot_model

plot_model(model, to_file='model1.png')

In [ ]:
from keras.utils import plot_model

plot_model(model, show_shapes=True, to_file='model1.png')